In [1]:
import django, os, sys, time, resource, re, gc, shutil
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from django.db.models import Count, Avg
from multiprocess import Pool
from functools import partial
import numpy as np
from functools import partial
from scipy.sparse import coo_matrix, csr_matrix, find, tril
import networkx as nx
import csv

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.models import *
qid = 1457

In [2]:
runs = [282, 283, 284, 285, 287]


fname = "../../data/topic_data/run_{}.xlsx".format(runs[0])

writer = pd.ExcelWriter(fname)

for i in range(1,len(runs)):

    topics1 = DynamicTopic.objects.filter(run_id=runs[i-1])
    topics2 = DynamicTopic.objects.filter(run_id=runs[i])

    df = pd.DataFrame.from_dict(list(topics2.values('title','id')))
    df['similarity'] = 0
    df['stopic'] = ""

    for t in topics2:
        scores = [0]
        titles = [""]
        for ct in topics1:
            score = len(set(t.top_words).intersection(set(ct.top_words)))
            if score>0:
                scores.append(score)
                titles.append(ct.title)

        
        m = max(scores)
        df.loc[df['id']==t.id,'similarity'] = m
        df.loc[df['id']==t.id,'stopic'] = titles[scores.index(max(scores))]


    df = df.sort_values('similarity',ascending=False)
    df.head()
    
    
    

    
    df.to_excel(writer,"run_{}".format(runs[i]))
    
writer.save()

In [3]:
runs = list(range(295,300))

writer = pd.ExcelWriter(fname)
fname = "../../data/topic_data/run_{}.xlsx".format(runs[0])

for i in range(1,len(runs)):

    topics1 = Topic.objects.filter(run_id=runs[i-1])
    topics2 = Topic.objects.filter(run_id=runs[i])

    df = pd.DataFrame.from_dict(list(topics2.values('title','id')))
    df['similarity'] = 0
    df['stopic'] = ""

    for t in topics2:
        scores = [0]
        titles = [""]
        for ct in topics1:
            score = len(set(t.top_words).intersection(set(ct.top_words)))
            if score>0:
                scores.append(score)
                titles.append(ct.title)

        
        m = max(scores)
        df.loc[df['id']==t.id,'similarity'] = m
        df.loc[df['id']==t.id,'stopic'] = titles[scores.index(max(scores))]


    df = df.sort_values('similarity',ascending=False)
    df.head()
    
    
    
    
    

    
    df.to_excel(writer,"run_{}".format(runs[i]))
    
writer.save()

In [32]:
runs = list(range(301,307))

#runs = [295,296,297]

col1s = []
col2s = []
ss = []
scols = []

for i in range(1,len(runs)):

    topics1 = Topic.objects.filter(run_id=runs[i-1])
    topics2 = Topic.objects.filter(run_id=runs[i])
    

    df = pd.DataFrame.from_dict(list(topics2.values('title','score')))
    
    df2 = pd.DataFrame.from_dict([{'title': 'None','score': 0}])
    
    df = df.append(df2)

    
    
    col1 = "topics_{}".format(runs[i-1])
    
    scol = "scores_{}".format(runs[i])
    bscol = "scores_{}".format(runs[i-1])
    
    if i==1:
        scols.append(bscol)
    
    col1s.append(col1)
    scols.append(scol)
    
    col2 = "topics_{}".format(runs[i])
    
    col2s.append(col2) 
    
    s = "similarity_{}-{}".format(runs[i-1],runs[i])
    ss.append(s)
    
    df = df.rename(columns = {'title': col2, 'score': scol})

    df[s] = 0
    df[col1] = "None"
    df[bscol] = 0

    for t in topics2:
        scores = [0]
        titles = [""]
        tscores = [0]
        for ct in topics1:
            score = len(set(t.top_words).intersection(set(ct.top_words)))
            if score>0:
                scores.append(score)
                titles.append(ct.title)
                tscores.append(ct.score)

        
        m = max(scores)
        df.loc[df[col2]==t.title, s] = m
        if m==0:
            df.loc[df[col2]==t.title, col1] = 'None'
        else:
            df.loc[df[col2]==t.title, col1] = titles[scores.index(max(scores))]
            df.loc[df[col2]==t.title, bscol] = tscores[scores.index(max(scores))]
            

    if i==1:   
        #df = pd.DataFrame.from_dict(list(topics2.values('title')))
        mdf = df
    else:
        mdf = mdf.merge(df,how="outer").fillna("")
    

columns = []
for i in range(len(col1s)):
    columns.append(col1s[i])
    columns.append(scols[i])
    columns.append(ss[i])
    if i == len(col1s)-1:
        columns.append(col2s[i])
        columns.append(scols[i+1])

print(columns)

mdf = mdf[columns]

mdf.head()



['topics_301', 'scores_301', 'similarity_301-302', 'topics_302', 'scores_302', 'similarity_302-303', 'topics_303', 'scores_303', 'similarity_303-304', 'topics_304', 'scores_304', 'similarity_304-305', 'topics_305', 'scores_305', 'similarity_305-306', 'topics_306', 'scores_306']


,topics_301,scores_301,similarity_301-302,topics_302,scores_302,similarity_302-303,topics_303,scores_303,similarity_303-304,topics_304,scores_304,similarity_304-305,topics_305,scores_305,similarity_305-306,topics_306,scores_306
0,"{plant, growth, root}",62.447042,6,"{plant, growth, root}",42.150649,9,"{plant, root, stress}",29.3845,8,"{plant, root, stress}",54.5742,8,"{plant, root, stress}",19.7081,7,"{plant, root, medicin}",18.1901
1,"{plant, growth, root}",62.447042,6,"{plant, growth, root}",42.150649,9,"{plant, root, stress}",29.3845,8,"{plant, root, stress}",54.5742,8,"{plant, root, stress}",19.7081,3,"{gene, resist, diseas}",15.7966
2,"{china, growth, econom}",89.103673,10,"{china, growth, industri}",64.565867,3,"{develop, countri, sustain}",29.4608,8,"{develop, countri, resourc}",22.8286,4,"{countri, global, nation}",106.237,,,
3,"{china, growth, econom}",89.103673,10,"{china, growth, industri}",64.565867,3,"{develop, countri, sustain}",29.4608,8,"{develop, countri, resourc}",22.8286,4,"{develop, sustain, goal}",18.9591,6,"{develop, countri, global}",43.2019
4,"{china, growth, econom}",89.103673,10,"{china, growth, industri}",64.565867,2,"{industri, china, sector}",100.788,8,"{industri, china, sector}",97.3522,3,"{region, china, resourc}",51.0362,5,"{china, chines, provinc}",76.5492


['topics_295', 'scores_295', 'similarity_295-296', 'topics_296', 'scores_296', 'similarity_296-297', 'topics_297', 'scores_297', 'similarity_297-298', 'topics_298', 'scores_298', 'similarity_298-299', 'topics_299', 'scores_299']


,topics_295,scores_295,similarity_295-296,topics_296,scores_296,similarity_296-297,topics_297,scores_297,similarity_297-298,topics_298,scores_298,similarity_298-299,topics_299,scores_299
0,"{research, social, develop}",146.796145,4,"{research, review, develop}",48.458977,8,"{research, review, literatur}",112.354202,8,"{research, review, literatur}",56.1284,5,"{review, discuss, challeng}",120.055
1,"{research, social, develop}",146.796145,4,"{research, review, develop}",48.458977,8,"{research, review, literatur}",112.354202,8,"{research, review, literatur}",56.1284,5,"{research, knowledg, scienc}",31.6434
2,"{research, social, develop}",146.796145,4,"{research, review, develop}",48.458977,2,"{social, econom, sustain}",38.445109,5,"{social, econom, dimens}",16.9183,6,"{social, econom, sustain}",11.5546
3,"{energi, renew, consumpt}",139.355578,8,"{energi, renew, consumpt}",86.187599,9,"{energi, renew, consumpt}",59.777449,1,"{effici, improv, exergi}",94.3624,10,"{effici, improv, exergi}",54.7991
4,"{energi, renew, consumpt}",139.355578,8,"{energi, renew, consumpt}",86.187599,9,"{energi, renew, consumpt}",59.777449,8,"{energi, renew, sourc}",45.2531,4,"{renew, sourc, generat}",6.67888


In [31]:
#col1s.append(col2)


#mdf = mdf.sort_values(columns,ascending=False).reset_index()


res = mdf.groupby(columns)



res = res.apply(lambda x: x.sort_values(s,ascending=False))

l = len(res)




fname = "../../data/topic_data/run_compare_{}.xlsx".format(runs[0])

writer = pd.ExcelWriter(fname, engine='xlsxwriter')

res.to_excel(writer)

worksheet = writer.sheets['Sheet1']

for i in range(len(ss)):
    c = chr(ord('A')-1+(i+1)*3)
    r = "{}2:{}{}".format(c,c,len(res))
    
    worksheet.conditional_format(r, {'type': '3_color_scale'})

writer.save()

res.head() 
    
    
res.head()

topics_295  \
topics_295               scores_295 similarity_295-296 topics_296                  scores_296 similarity_296-297 topics_297                  scores_297 similarity_297-298 topics_298                   scores_298 similarity_298-299 topics_299                   scores_299                                 
None                     0.0000     0                  None                        0.000000   0                  None                        0.000000   0.0                None                         0.000000   0.0                None                         0.000000   130                      None   
                                                                                                                                                                                                                                      }                            0.000000   129                      None   
{build, design, thermal} 105.2683   4                  {heat, thermal, temperatur} 49.750279  9                  {heat, thermal, temperatur} 40.863319  3.0                {temperatur, degre, thermal} 94.027477  9.0                {temperatur, degre, increas} 100.014052 46   {build, design, thermal}   
                                                                                                                                                        7.0                {heat, thermal, cool}        15.449264  3.0                {thermal, comfort, insul}    52.575082  44   {build, design, thermal}   
                                                                                                                                                                                                                   7.0                {heat, cool, transfer}       6.724830   45   {build, design, thermal}   

                                                                                                                                                                                                                                                                                   scores_295  \
topics_295               scores_295 similarity_295-296 topics_296                  scores_296 similarity_296-297 topics_297                  scores_297 similarity_297-298 topics_298                   scores_298 similarity_298-299 topics_299                   scores_299                   
None                     0.0000     0                  None                        0.000000   0                  None                        0.000000   0.0                None                         0.000000   0.0                None                         0.000000   130      0.0000   
                                                                                                                                                                                                                                      }                            0.000000   129      0.0000   
{build, design, thermal} 105.2683   4                  {heat, thermal, temperatur} 49.750279  9                  {heat, thermal, temperatur} 40.863319  3.0                {temperatur, degre, thermal} 94.027477  9.0                {temperatur, degre, increas} 100.014052 46     105.2683   
                                                                                                                                                        7.0                {heat, thermal, cool}        15.449264  3.0                {thermal, comfort, insul}    52.575082  44     105.2683   
                                                                                                                                                                                                                   7.0                {heat, cool, transfer}       6.724830   45     105.2683   

                                                                                                                                               

In [ ]:
print(mdf)

In [ ]:
runs = list(range(295,300))

runs = []

for i in range(len(runs)):

    topics = Topic.objects.filter(run_id=runs[i])
    
    tree = []
    
    for t in topics:
        
        tdict = {
            "parent_similarity": 0,
            "run": runs[i],
            "ntopics": topics.count(),
            "title": t.title,
            "sim_topics": [],
            "child_topics": 0
        }
        
        if i==0:
            tree.append(tdict)
        else:
            
            scores = [0]
            titles = [""]
            for ct in comparison_topics:
                score = len(set(t.top_words).intersection(set(ct.top_words)))
                if score>0:
                    scores.append(score)
                    titles.append(ct.title)


                parent = titles[scores.index(max(scores))]
                
                
                
                print(m)

In [ ]:
def find_similar(x):
    return(1,2)

df['bla'], df['x'] = zip(df.map(find_similar))
                         
df.head()